In [7]:
import numpy as np
from PlotSettings import MainColor1, plt
from CrossSections import Transfer_Sigma_Low_Energy, Transfer_Sigmav_Low_Energy, sigv_T

## Fits con datos de Yu

In [4]:

#for the plots
DwarfData = np.loadtxt('Data-Sets/DwarfDataPlot.csv', delimiter='\t')
Dwarf_x =     DwarfData[:,0]
Dwarf_y =     DwarfData[:,1]
Dwarf_Err_l = DwarfData[:,2]
Dwarf_Err_r = DwarfData[:,3]
Dwarf_Err_d = DwarfData[:,4]
Dwarf_Err_u = DwarfData[:,5]

GalaxyData = np.loadtxt('Data-Sets/GalaxyDataPlot.csv', delimiter='\t')
Galaxy_x = GalaxyData[:,0]
Galaxy_y = GalaxyData[:,1]
Galaxy_Err_l = GalaxyData[:,2]
Galaxy_Err_r = GalaxyData[:,3]
Galaxy_Err_d = GalaxyData[:,4]
Galaxy_Err_u = GalaxyData[:,5]

ClusterData = np.loadtxt('Data-Sets/ClusterDataPlot.csv', delimiter='\t')
Cluster_x = ClusterData[:,0]
Cluster_y = ClusterData[:,1]
Cluster_Err_l = ClusterData[:,2]
Cluster_Err_r = ClusterData[:,3]
Cluster_Err_d = ClusterData[:,4]
Cluster_Err_u = ClusterData[:,5]

# For the fit
OrderedData = np.loadtxt('Data-Sets/Datos_ordenados.csv')
x_data = OrderedData[:,0]
y_data = OrderedData[:,1]
err_x_data = OrderedData[:,2]
err_y_data = OrderedData[:,3]


In [6]:
from scipy.optimize import minimize

alpha_EM = 1/137
e = np.sqrt(4*np.pi*alpha_EM)


def compute_chi2(free_params, x=None, data=None, err=None):
    
    M, m = free_params
    errx, erry = err
    model = [Transfer_Sigmav_Low_Energy(x, g=e, M=M, m=m) for x in x]

    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2x + chi2y

priors = [0.02, 10]

Sol = minimize(compute_chi2, priors, args=(x_data, y_data, (err_x_data, err_y_data)) )

Sol.x



array([ 0.02, 10.  ])

## Fits con Datos de Camila

In [14]:
#Import Data from Camila Correa
CamilaData = np.loadtxt('Data-Sets/Data_Fig7_Correa_2021.txt')

VelocityData = CamilaData[:,0]
VelocityData_left_Err = CamilaData[:,1]
VelocityData_right_Err = CamilaData[:,2]

WightedCrossSectionData = CamilaData[:,3]
WightedCrossSectionData_left_Err = CamilaData[:,4]
WightedCrossSectionData_right_Err = CamilaData[:,5]


In [8]:
# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=None, data=None, err=None):
    #Compute model
    lg, lM, lm = free_params
    g, M, m = pow(10,lg), pow(10, lM), pow(10, lm)
    model = [sigv_T(x, g, M, m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2x + chi2y

from scipy.optimize import minimize


g1, M1, m1 = 3e-2, 5e-3, 5
l_g1 = np.log10(g1)
l_M1 = np.log10(M1)
l_m1 = np.log10(m1)

initial = [l_g1, l_M1, l_m1]


soln = minimize(compute_chi2, initial, args=(x_data, y_data, (err_x_data, err_y_data)) )

log_g_ml, log_M_ml, log_m_ml = soln.x

print("Maximum likelihood estimates:")
print("log g = {0:.3f}".format(log_g_ml))
print("log M = {0:.3f}".format(log_M_ml))
print("log m = {0:.3f}".format(log_m_ml))

d:\Nicolás\Documents\Research\Pre-Prints\Leptophilic-Atlas\Leptophilic-Atlas-Repo\Leptophilic-Atlas-Repo\CrossSections.py:70: RuntimeWarning: invalid value encountered in log
  sigma_t = Prefactor*( (s*v**2*(2*s + 3*M**2)+2*(M**2 + 2*m**2)**2)/(2*M**2*(M**2+s*v**2)) - (M**2 + s)/(s*v**2)*np.log(1 + s*v**2/M**2) )
d:\Nicolás\Documents\Research\Pre-Prints\Leptophilic-Atlas\Leptophilic-Atlas-Repo\Leptophilic-Atlas-Repo\CrossSections.py:72: RuntimeWarning: invalid value encountered in log
  sigma_st = -Prefactor/2*( (16*m**2+2*M**2+ 3*s*v**2)/(s-M**2) - (4*m**2 + 2*m**2*(4*M**2+3*s*v**2+s ) + (M**2+s*v**2)**2 )/(s*v**2*(s-M**2))*np.log(1 + s*v**2/M**2) )
d:\Nicolás\Documents\Research\Pre-Prints\Leptophilic-Atlas\Leptophilic-Atlas-Repo\Leptophilic-Atlas-Repo\CrossSections.py:62: RuntimeWarning: invalid value encountered in log
  return  fc*Prefactor*(Term1 + LogTerm1*np.log(1 + s*v**2/M**2) )
d:\Nicolás\Documents\Research\Pre-Prints\Leptophilic-Atlas\Leptophilic-Atlas-Repo\Leptophilic-Atlas

Maximum likelihood estimates:
log g = -1.523
log M = -2.301
log m = 0.699
